In [ ]:
%load_ext autoreload

# Кастомная библиотека, созданная для хакатона - модкли лежат в папке src
from bank_schedule import dataclaster, optim, output, check_conditions


# Загрузка данных

In [ ]:
# В эту папку требуется положить 3 файла:
# Params.xlsx - набор основных констант(лежит в bank_schedule/data/raw)
# terminal_data_hackathon v4.xlsx - Как в задании
# times v4.csv - Как в задании
PATH_TO_DATA_FOLDER = '/Users/sykuznetsov/Documents/GitHub/bank_schedule/data/raw'

# Интерфейс для доступа к данным
data = dataclaster.DataClaster(PATH_TO_DATA_FOLDER)

# Определение количества машин

## Прогноз

## Определние кол-ва машин

In [ ]:
cars_quant = 6

# Эвристическая оптимизация

# Подготовка отчета по эвристической оптимизации

In [ ]:
df_routes = output.find_routes_check_2(obj, data) # проверка ограничений на время (с 8 утра до 8 вечера) + составления маршрутов для отчета
df_opt_result = df_routes[['TID','date','auto']]

check_conditions.condition_max_days_inc(df_opt_result,  data)
check_conditions.check_over_balance(df_opt_result,  data)

df_money_sum = output.find_ATM_balance_evening(df_opt_result, data) 
df_money_sum_morning = output.find_ATM_balance_morning(df_opt_result, data) 
all_cost_by_days, all_cost_inc, df_fond = output.find_all_cost(df_money_sum, df_opt_result, data) 

# создание отчета
output.postprocessing_and_make_excel('result_6_clusters.xlsx', df_money_sum_morning, df_fond, all_cost_inc, all_cost_by_days, df_routes)

# Кластеризация

In [ ]:
data.run_cluster(0.01, cars_quant)

# Каринка кластеризации

# MILP оптимизация

In [ ]:
%autoreload 2

# Решение может искаться долго(до 6 часов),но зато показывает в средним результаты лучше, чем у эвристической оптимизации

milp_optim_results = []
for cluster_num in range(cars_quant):
    presolved_models, presolved_results = optim.run_milp_optim(data, date(2022, 9, 1), 2, cluster_num)
    milp_optim_results.append((presolved_models, presolved_results, cluster_num))


# Подготовка отчета по MILP оптимизации

## Проверка ограничений

In [ ]:
# Вынимаем маршруты из внутреннего представления
# Проверка ограничений на время (с 8 утра до 8 вечера) + составления маршрутов для отчета
df_routes = output.find_routes_check(milp_optim_results, data)
df_opt_result = df_routes[['TID','date','auto']]

# Проверка ограничения на инкассации каждые 14 дней
check_conditions.condition_max_days_inc(df_opt_result,  data)

# Проверка ограничения на максимальную допустимую сумму в банкоматах
check_conditions.check_over_balance(df_opt_result,  data)

## Подготовка отчета

In [ ]:
%autoreload 2
# Создание отчета
df_money_sum = output.find_ATM_balance_evening(df_opt_result, data) 
df_money_sum_morning = output.find_ATM_balance_morning(df_opt_result, data) 
all_cost_by_days, all_cost_inc, df_fond = output.find_all_cost(df_money_sum, df_opt_result, data) 

output.postprocessing_and_make_excel('MILP optimisation.xlsx', df_money_sum_morning, df_fond, all_cost_inc, all_cost_by_days, df_routes)